In [1]:
import nltk
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
train_data_raw = pd.read_csv('train_2kmZucJ.csv')
test_data_raw=pd.read_csv('test_oJQbWVk.csv')

In [5]:
train_data_raw.head(10)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
5,6,1,iPhone software update fucked up my phone big ...
6,7,0,Happy for us .. #instapic #instadaily #us #son...
7,8,0,New Type C charger cable #UK http://www.ebay.c...
8,9,0,Bout to go shopping again listening to music #...
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...


In [10]:
#first of all preprocess the data
import re
import string

stop_words = stopwords.words("english")
def text_preproc(x):
  x = x.lower()
  # x = ' '.join(wordnet.lemmatize(word, 'v') for word in x.split())
  x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  x = x.encode('ascii', 'ignore').decode()
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#\S+', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x
train_data_raw['clean_text'] = train_data_raw.tweet.apply(text_preproc)
test_data_raw['clean_text'] = test_data_raw.tweet.apply(text_preproc)

In [11]:
train_data_raw['clean_text'].head(10)

0                                                test 
1       finally transparant silicon case thanks uncle 
2                                  love this would go 
3                      i wired know i george made way 
4    amazing service apple even talk question unles...
5    iphone software update fucked phone big time s...
6                                            happy us 
7    new type c charger cable new year cross young ...
8                    bout go shopping listening music 
9                                               photo 
Name: clean_text, dtype: object

# Build a Term-Document Matrix with TF-IDF 

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_data_raw['clean_text']).toarray()
df_new = pd.DataFrame(X, columns=vectorizer.get_feature_names())
X_test = vectorizer.transform(test_data_raw['clean_text']).toarray()
test_new = pd.DataFrame(X_test, columns=vectorizer.get_feature_names())

In [14]:
df_new.head()

,aa,aaaahhhhhhh,aah,aalborg,aand,aapl,aaron,aarrrggghhhh,aashamsakal,aaydojbfkq,ab,abah,abi,ability,abit,able,aboard,about,absence,absolute,absolutely,absurd,abt,abukamalyasinpic,abuse,academic,academy,acc,accelerate,accept,acceptable,accepted,accepts,access,accessed,accessible,accessories,accessory,accident,accidental,...,youtube,youve,yr,yrs,yt,yujujiioioo,yumm,yummmmy,yummy,yup,yureka,yvopwgpesb,yvtftpawnk,za,zachary,zain,zap,zapio,zazzle,zealand,zebra,zeeland,zen,zepeihszmu,zero,zeta,zip,zipper,zj,znjq,zombies,zonbies,zone,zoo,zoom,zoooom,zune,zunehd,zvgbxciqwk,zyrithxkuv
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
X = df_new.values
y = train_data_raw.label.values
kfold = KFold(n_splits=10)
# Define the model
nb_multinomial = MultinomialNB()
nb_bernoulli = BernoulliNB()
# As a storage of the model's performance
def calculate_f1(model):
 metrics = []
 
 for train_idx, test_idx in kfold.split(X):
   X_train, X_test = X[train_idx], X[test_idx]
   y_train, y_test = y[train_idx], y[test_idx]
   model.fit(X_train, y_train)
   y_pred = model.predict(X_test)
   metrics.append(f1_score(y_test, y_pred))
 
 # Retrieve the mean of the result
 print("%.3f" % np.array(metrics).mean())


In [19]:
calculate_f1(nb_bernoulli)

0.733
